## 인간 개입형 패턴으로 함수 도구를 사용하기

에이전트가 함수 호출 승인과 같이 사용자 입력을 필요로 하는 경우, 이를 '인간 개입형(human-in-the-loop)' 패턴이라고 합니다. 사용자 입력이 필요한 에이전트 실행은 최종 답변을 제공하는 대신, 사용자에게 필요한 입력이 무엇인지 알려주는 응답으로 완료됩니다. 에이전트를 호출한 측에서는 사용자로부터 필요한 입력을 받아 새로운 에이전트 실행의 일부로 다시 에이전트에 전달해야 합니다.

In [10]:
from dotenv import load_dotenv
load_dotenv()  # .env 파일 로드

True

#### 승인이 필요한 Function 도구를 사용하여 에이전트를 생성

함수를 사용할 때, 각 함수가 실행되기 전에 사람의 승인이 필요한지 여부를 지정할 수 있습니다. 이는 @ai_function 데코레이터를 사용할 때 approval_mode 매개변수를 "always_require"f로 설정함으로써 가능합니다.

다음은 특정 위치의 날씨 정보를 가져오는 것처럼 보이게 하는 간단한 함수 도구의 예입니다.

In [1]:
from typing import Annotated
from agent_framework import ai_function

@ai_function
def get_weather(location: Annotated[str, "The city and state, e.g. San Francisco, CA"]) -> str:
    """Get the current weather for a given location."""
    return f"The weather in {location} is cloudy with a high of 15°C."

In [2]:
from typing import Annotated
from agent_framework import ai_function

@ai_function(approval_mode="always_require")
def get_weather_detail(location: Annotated[str, "The city and state, e.g. San Francisco, CA"]) -> str:
    """Get detailed weather information for a given location."""
    return f"The weather in {location} is cloudy with a high of 15°C, humidity 88%."

이제 에이전트를 생성할 때, ChatAgent 생성자에 도구 목록을 전달하여 승인이 필요한 function 도구를 에이전트에 제공할 수 있습니다.

In [18]:
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIResponsesClient
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

agent = ChatAgent(
    chat_client=AzureOpenAIChatClient(credential=AzureCliCredential()),
    name="WeatherAgent",
    instructions=(
        "You are a helpful weather assistant."
    ),
    tools=[get_weather, get_weather_detail],
)
    
# 에이전트 실행
response = await agent.run("What's the weather like in New York?")
print(response.text)

The weather in New York is currently cloudy, with a high temperature of 15°C. If you need more detailed weather information, just let me know!


이제 승인이 필요한 function 도구를 사용하는 호출을 수행해 봅니다.

In [15]:
result = await agent.run("What is the detailed weather like in Seoul?")

if result.user_input_requests:
    for user_input_needed in result.user_input_requests:
        print(f"Function: {user_input_needed.function_call.name}")
        print(f"Arguments: {user_input_needed.function_call.arguments}")

Function: get_weather_detail
Arguments: {"location":"Seoul"}


이제 승인이 필요한 함수가 있으므로, 에이전트는 함수를 직접 실행하고 결과를 반환하는 대신 승인 요청을 보낼 수 있습니다. 응답에서 user_input_requests이 있는지 확인하면 에이전트가 함수 실행에 사용자 승인을 요구하는지 알 수 있습니다.

함수 승인 요청이 있는 경우, 함수 이름과 인수를 포함한 함수 호출 세부 정보는 사용자 입력 요청의 function_call 속성에서 확인할 수 있습니다. 이 정보를 사용자에게 보여주면 사용자는 함수 호출을 승인할지 거부할지 결정할 수 있습니다.

사용자가 입력을 완료하면 사용자 입력 요청의 create_response 메서드를 사용하여 응답을 생성할 수 있습니다. 함수 호출을 승인하려면 True 를 전달하고, 거부하려면 False를 전달하세요.

그러면 응답을 새로운 ChatMessage로 에이전트에 전달하여 에이전트로부터 결과를 다시 받을 수 있습니다.

In [ ]:
## 예시 코드입니다.
## 실제 코드는 다음 영역에서 계속됩니다.
from agent_framework import ChatMessage, Role

# Get user approval (in a real application, this would be interactive)
user_approval = True  # or False to reject

# Create the approval response
approval_message = ChatMessage(
    role=Role.USER, 
    contents=[user_input_needed.create_response(user_approval)]
)

# Continue the conversation with the approval
final_result = await agent.run([
    "What is the detailed weather like in Amsterdam?",
    ChatMessage(role=Role.ASSISTANT, contents=[user_input_needed]),
    approval_message
])
print(final_result.text)

승인 절차를 반복적으로 처리합니다.
승인이 필요한 여러 함수 호출을 처리할 때는 모든 함수가 승인되거나 거부될 때까지 반복문을 사용하여 승인을 처리해야 할 수 있습니다.

In [19]:
async def handle_approvals(query: str, agent) -> str:
    """Handle function call approvals in a loop."""
    current_input = query

    while True:
        result = await agent.run(current_input)

        if not result.user_input_requests:
            # No more approvals needed, return the final result
            return result.text

        # Build new input with all context
        new_inputs = [query]

        for user_input_needed in result.user_input_requests:
            print(f"Approval needed for: {user_input_needed.function_call.name}")
            print(f"Arguments: {user_input_needed.function_call.arguments}")

            # Add the assistant message with the approval request
            new_inputs.append(ChatMessage(role=Role.ASSISTANT, contents=[user_input_needed]))

            # Get user approval (in practice, this would be interactive)
            user_approval = True  # Replace with actual user input

            # Add the user's approval response
            new_inputs.append(
                ChatMessage(role=Role.USER, contents=[user_input_needed.create_response(user_approval)])
            )

        # Continue with all the context
        current_input = new_inputs

# Usage
result_text = await handle_approvals("Get detailed weather for Seattle and Portland", agent)
print(result_text)

Approval needed for: get_weather_detail
Arguments: {"location": "Seattle, WA"}
Approval needed for: get_weather_detail
Arguments: {"location": "Portland, OR"}
Here are the detailed weather conditions:

- Seattle, WA: Cloudy, high of 15°C, humidity 88%.
- Portland, OR: Cloudy, high of 15°C, humidity 88%.

Let me know if you need more details or forecasts for these cities!
